# Vector Design Prototype
## Written by Jim Pivarski, with Henry Schreiner

In [1]:
import sys
import operator
import json
import numpy as np
import numba as nb

import uproot
import awkward1 as ak
from skhep_testdata import data_path

This is an free-standing illustration of the original design of Vector and the integration with Awkward1.

### Import data
Get some data from Uproot via scikit-hep-testdata

In [2]:
tree = uproot.open(data_path("uproot-HZZ.root"))["events"]

In [3]:
x, y, z, t = tree.arrays(["Muon_Px", "Muon_Py", "Muon_Pz", "Muon_E"], outputtype=tuple)
offsets = ak.layout.Index64(x.offsets)
content = ak.layout.RecordArray({
    "x": ak.layout.NumpyArray(x.content.astype(np.float64)),
    "y": ak.layout.NumpyArray(y.content.astype(np.float64)), 
    "z": ak.layout.NumpyArray(z.content.astype(np.float64)),
    "t": ak.layout.NumpyArray(t.content.astype(np.float64))},
    parameters={"__record__": "LorentzXYZT"})

I like `LorentzXYZT` as a name for Cartesian Lorentz vectors. It can recognizably
be shortened to `Lxyz` and it invites the cylindrical form to be `LorentzCyl`/`Lcyl`.

They should be interchangeable: having the same methods/properties and freely
returning whichever form is most convenient. Source vectors would likely be `Lcyl`
and adding them would likely return `Lxyz`, for instance.

This array is generic: it doesn't know what records labeled `LorentzXYZT` mean.

In [4]:
example = ak.Array(ak.layout.ListOffsetArray64(offsets, content))
print(repr(example))

<Array [[{x: -52.9, y: -11.7, ... t: 69.6}]] type='2421 * var * LorentzXYZT["x":...'>


### Free functions

These functions can be reused for LorentzXYZT objects, LorentzXYZTArray arrays, and Numba.

In [5]:
def lorentz_xyz_pt(rec):
    return np.sqrt(rec.x**2 + rec.y**2)

def lorentz_xyz_eta(rec):
    return np.arcsinh(rec.z / np.sqrt(rec.x**2 + rec.y**2))

def lorentz_xyz_phi(rec):
    return np.arctan2(rec.y, rec.x)

def lorentz_xyz_mass(rec):
    return np.sqrt(rec.t**2 - rec.x**2 - rec.y**2 - rec.z**2)

This function only works as a ufunc overload

In [6]:
def lorentz_add_xyz_xyz(left, right):
    x = left.x + right.x
    y = left.y + right.y
    z = left.z + right.z
    t = left.t + right.t

    return ak.zip({"x": x, "y": y, "z": z, "t": t},
                  with_name="LorentzXYZT")

### Mixin class

Many of the functions can be used for records and arrays of them, so use a base class. This is a *mixin* - no **members** or **constructors** allowed.

In [7]:
class LorentzXYZTCommon(object):
    @property
    def pt(self):
        with np.errstate(invalid="ignore"):
            return lorentz_xyz_pt(self)

    @property
    def eta(self):
        with np.errstate(invalid="ignore"):
            return lorentz_xyz_eta(self)

    @property
    def phi(self):
        with np.errstate(invalid="ignore"):
            return lorentz_xyz_phi(self)

    @property
    def mass(self):
        with np.errstate(invalid="ignore"):
            return lorentz_xyz_mass(self)

### Awkward behavior

This is all that is required to make an Awkward array behavior:

In [8]:
class LorentzXYZT(ak.Record, LorentzXYZTCommon):
    def __repr__(self):
        return "Lxyz({0:.3g} {1:.3g} {2:.3g} {3:.3g})".format(self.x, self.y, self.z, self.t)

class LorentzXYZTArray(ak.Array, LorentzXYZTCommon):
    pass

Now, we need a behavior dict:

In [9]:
# Define some behaviors for Lorentz vectors.
lorentzbehavior = dict(ak.behavior)

# Any records with __record__ = "LorentzXYZT" will be mapped to LorentzXYZT instances.
lorentzbehavior["LorentzXYZT"] = LorentzXYZT

# Any arrays containing such records (any number of levels deep) will be LorentsXYZArrays.
lorentzbehavior["*", "LorentzXYZT"] = LorentzXYZTArray

# The NumPy ufunc for "add" will use our definition for __record__ = "LorentzXYZT".
lorentzbehavior[np.add, "LorentzXYZT", "LorentzXYZT"] = lorentz_add_xyz_xyz

### Example with Awkward

This is ready to be used:

In [10]:
# This new array understands that data labeled "LorentzXYZT" should have the above methods.
example2 = ak.Array(example, behavior=lorentzbehavior)

print(repr(example2))
print(repr(example2[0, 0]))
print(repr(example2[0, 0].mass))
print(repr(example2.mass))

<LorentzXYZTArray [[Lxyz(-52.9 -11.7 -8.16 54.8), ... ] type='2421 * var * Lorent...'>
Lxyz(-52.9 -11.7 -8.16 54.8)
0.10559298741436905
<Array [[0.106, 0.105], ... [0.104]] type='2421 * var * float64'>


In [11]:
example.x

<Array [[-52.9, 37.7], ... 1.14], [23.9]] type='2421 * var * float64'>

In [12]:
# We need a "ak.sizes" function with a simpler interface than this...
hastwo = ak.count(example2, axis=-1).x >= 2

print(example2[hastwo, 0] + example2[hastwo, 1])
print((example2[hastwo, 0] + example2[hastwo, 1]).mass)

[Lxyz(-15.2 -11 -19.5 94.2), Lxyz(49.8 8.08 48.1 102), ... Lxyz(2.94 18.4 -262 273)]
[90.2, 74.7, 89.8, 94.9, 92.1, 53.4, 89.8, ... 91.7, 88.8, 101, 91.5, 92.1, 85.4, 76]


## Numba
### Numba: Simple Awkward property access


Now we need typers and lowerers:

In [13]:
def typer_lorentz_xyz_pt(viewtype):
    return nb.float64

def lower_lorentz_xyz_pt(context, builder, sig, args):
    return context.compile_internal(builder, lorentz_xyz_pt, sig, args)

These can be awkward behaviors:

In [14]:
lorentzbehavior["__numba_typer__", "LorentzXYZT", "pt"] = typer_lorentz_xyz_pt
lorentzbehavior["__numba_lower__", "LorentzXYZT", "pt"] = lower_lorentz_xyz_pt

If we wanted a method (with arguments determined in the typer), the signature would be:

```python
lorentzbehavior["__numba_lower__", "LorentzXYZT", "pt", ()] = ...
```

#### Example

Let's try accessing this in Numba:

In [15]:
example3 = ak.Array(example, behavior=lorentzbehavior)

@nb.njit
def do_it_in_numba(input, output):
    for event in input:
        output.begin_list()

        for muon in event:
            output.begin_record()
            output.field("muon")
            output.append(muon)
            output.field("pt")
            output.append(muon.pt)
            output.end_record()

        output.end_list()

output = ak.ArrayBuilder(behavior=lorentzbehavior)
do_it_in_numba(example3, output)

print(output.snapshot())

[[{muon: Lxyz(-52.9 -11.7 -8.16 54.8), pt: 54.2}, ... pt: 42.9}]]


### Numba: binary operators

We can define binary operations (operator.add being the one we want most)...

In [16]:
def typer_lorentz_xyz_eq(binop, left, right):
    return nb.boolean(left, right)

def lower_lorentz_xyz_eq(context, builder, sig, args):
    def compute(left, right):
        return abs(left.x - right.x) + abs(left.y - right.y) + abs(left.z - right.z) + abs(left.t - right.t) < 0.001
    return context.compile_internal(builder, compute, sig, args)

lorentzbehavior["__numba_typer__", "LorentzXYZT", operator.eq, "LorentzXYZT"] = typer_lorentz_xyz_eq
lorentzbehavior["__numba_lower__", "LorentzXYZT", operator.eq, "LorentzXYZT"] = lower_lorentz_xyz_eq

#### Example

In [17]:
example4 = ak.Array(example, behavior=lorentzbehavior)

@nb.njit
def check_equality(input, output):
    for muons in input:
        output.begin_list()

        for i in range(len(muons)):
            output.begin_list()
            for j in range(i, len(muons)):
                output.append(muons[i] == muons[j])
            output.end_list()

        output.end_list()

output = ak.ArrayBuilder(behavior=lorentzbehavior)
check_equality(example4, output)

print(output.snapshot())

[[[True, False], [True]], [[True]], [[True, ... [[True]], [[True]], [[True]]]


### Aside: free vectors

The trouble with `operator.add` is that it returns new objects.

The records we have been dealing with are not free-floating objects; they're views
into the arrays, and Awkward Arrays can't be created in Numba (that restriction gives
us a lot of freedom and this model favors the development of a functional language).

So we need to create a new Numba type that is a free-floating `LorentzXYZTCommon`.
Fortunately, that's a more conventional task and serves as a good introduction to Numba.

In [18]:
class LorentzXYZTFree(LorentzXYZTCommon):
    def __init__(self, x, y, z, t):
        self.x = x
        self.y = y
        self.z = z
        self.t = t

    def __repr__(self):
        return "Lxyz({0:.3g} {1:.3g} {2:.3g} {3:.3g})".format(self.x, self.y, self.z, self.t)

    def __getitem__(self, attr):
        # It has to behave the same way as the bound objects or users will get confused.
        if attr in ("x", "y", "z", "t"):
            return getattr(self, attr)
        else:
            raise ValueError("key {0} does not exist (not in record)".format(json.dumps(attr)))

### Numba: Extending free vectors

In [19]:
@nb.extending.typeof_impl.register(LorentzXYZTFree)
def typeof_LorentzXYZTFree(obj, c):
    return LorentzXYZTType()

class LorentzXYZTType(nb.types.Type):
    def __init__(self):
        # Type names have to be unique identifiers; they determine whether Numba
        # will recompile a function with new types.
        super(LorentzXYZTType, self).__init__(name="LorentzXYZTType()")

@nb.extending.register_model(LorentzXYZTType)
class LorentzXYZTModel(nb.extending.models.StructModel):
    def __init__(self, dmm, fe_type):
        # This is the C-style struct that will be used wherever LorentzXYZT are needed.
        members = [("x", nb.float64),
                   ("y", nb.float64),
                   ("z", nb.float64),
                   ("t", nb.float64)]
        super(LorentzXYZTModel, self).__init__(dmm, fe_type, members)

#### Unboxing and boxing:

In [20]:
@nb.extending.unbox(LorentzXYZTType)
def unbox_LorentzXYZT(lxyztype, lxyzobj, c):
    # How to turn LorentzXYZTFree Python objects into LorentzXYZTModel structs.
    x_obj = c.pyapi.object_getattr_string(lxyzobj, "x")
    y_obj = c.pyapi.object_getattr_string(lxyzobj, "y")
    z_obj = c.pyapi.object_getattr_string(lxyzobj, "z")
    t_obj = c.pyapi.object_getattr_string(lxyzobj, "t")

    # "values" are raw LLVM code; "proxies" have getattr/setattr logic to access fields.
    outproxy = c.context.make_helper(c.builder, lxyztype)

    # https://github.com/numba/numba/blob/master/numba/core/pythonapi.py
    outproxy.x = c.pyapi.float_as_double(x_obj)
    outproxy.y = c.pyapi.float_as_double(y_obj)
    outproxy.z = c.pyapi.float_as_double(z_obj)
    outproxy.t = c.pyapi.float_as_double(t_obj)

    # Yes, we're in that world...
    c.pyapi.decref(x_obj)
    c.pyapi.decref(y_obj)
    c.pyapi.decref(z_obj)
    c.pyapi.decref(t_obj)

    is_error = nb.core.cgutils.is_not_null(c.builder, c.pyapi.err_occurred())
    return nb.extending.NativeValue(outproxy._getvalue(), is_error)

@nb.extending.box(LorentzXYZTType)
def box_LorentzXYZT(lxyztype, lxyzval, c):
    # This proxy is initialized with a value, used for getattr, rather than setattr.
    inproxy = c.context.make_helper(c.builder, lxyztype, lxyzval)
    x_obj = c.pyapi.float_from_double(inproxy.x)
    y_obj = c.pyapi.float_from_double(inproxy.y)
    z_obj = c.pyapi.float_from_double(inproxy.z)
    t_obj = c.pyapi.float_from_double(inproxy.t)

    # The way we get Python objects into this lowered world is by pickling them.
    LorentzXYZTFree_obj = c.pyapi.unserialize(c.pyapi.serialize_object(LorentzXYZTFree))

    out = c.pyapi.call_function_objargs(LorentzXYZTFree_obj, (x_obj, y_obj, z_obj, t_obj))

    c.pyapi.decref(LorentzXYZTFree_obj)
    c.pyapi.decref(x_obj)
    c.pyapi.decref(y_obj)
    c.pyapi.decref(z_obj)
    c.pyapi.decref(t_obj)

    return out

#### Example

Now we've defined enough that our objects can go into and come out of Numba.

In [21]:
testit = LorentzXYZTFree(1, 2, 3, 4)
print(testit)

@nb.njit
def pass_through(obj):
    return obj

print(testit, pass_through(testit))

Lxyz(1 2 3 4)
Lxyz(1 2 3 4) Lxyz(1 2 3 4)


Notice that the original has int fields and the passed-through has floats:

In [22]:
print(testit.x, pass_through(testit).x)

1 1.0


### Numba: Constructors

Defining an in-Numba constructor is a separate thing.

In [23]:
@nb.extending.type_callable(LorentzXYZTFree)
def typer_LorentzXYZTFree_constructor(context):
    def typer(x, y, z, t):
        if x == nb.types.float64 and y == nb.types.float64 and z == nb.types.float64 and t == nb.types.float64:
            return LorentzXYZTType()
    return typer

@nb.extending.lower_builtin(LorentzXYZTFree, nb.types.float64, nb.types.float64, nb.types.float64, nb.types.float64)
def lower_LorentzXYZTFree_constructor(context, builder, sig, args):
    rettype, (xtype, ytype, ztype, ttype) = sig.return_type, sig.args
    xval, yval, zval, tval = args

    outproxy = context.make_helper(builder, rettype)
    outproxy.x = xval
    outproxy.y = yval
    outproxy.z = zval
    outproxy.t = tval

    return outproxy._getvalue()

#### Example

Test it!

In [24]:
@nb.njit
def test_constructor():
    return LorentzXYZTFree(1.1, 2.2, 3.3, 4.4)

print(test_constructor())

Lxyz(1.1 2.2 3.3 4.4)


### Numba: Methods and properties

Now it's time to define the methods and properties.

To simply map model attributes to user-accessible properties, use a macro.

In [25]:
nb.extending.make_attribute_wrapper(LorentzXYZTType, "x", "x")
nb.extending.make_attribute_wrapper(LorentzXYZTType, "y", "y")
nb.extending.make_attribute_wrapper(LorentzXYZTType, "z", "z")
nb.extending.make_attribute_wrapper(LorentzXYZTType, "t", "t")

For more general cases, there's an `AttributeTemplate`.

In [26]:
@nb.extending.infer_getattr
class typer_LorentzXYZT_methods(nb.core.typing.templates.AttributeTemplate):
    key = LorentzXYZTType

    def generic_resolve(self, lxyztype, attr):
        if attr == "pt":
            return nb.float64
        elif attr == "eta":
            return nb.float64
        elif attr == "phi":
            return nb.float64
        elif attr == "mass":
            return nb.float64
        else:
            # typers that return None defer to other typers.
            return None

If we had any methods with arguments, this is how we'd do it.

```python
@nb.typing.templates.bound_function("pt")
def pt_resolve(self, lxyztype, args, kwargs):
    ...
```

#### Lowering

To lower these functions, we can duck-type the Python functions above.
Since they're defined in terms of NumPy functions, they apply to

* Python scalars
* NumPy arrays
* Awkward arrays
* lowered Numba values

In [27]:
@nb.extending.lower_getattr(LorentzXYZTType, "pt")
def lower_LorentzXYZT_pt(context, builder, lxyztype, lxyzval):
    return context.compile_internal(builder, lorentz_xyz_pt, nb.float64(lxyztype), (lxyzval,))

@nb.extending.lower_getattr(LorentzXYZTType, "eta")
def lower_LorentzXYZT_eta(context, builder, lxyztype, lxyzval):
    return context.compile_internal(builder, lorentz_xyz_eta, nb.float64(lxyztype), (lxyzval,))

@nb.extending.lower_getattr(LorentzXYZTType, "phi")
def lower_LorentzXYZT_phi(context, builder, lxyztype, lxyzval):
    return context.compile_internal(builder, lorentz_xyz_phi, nb.float64(lxyztype), (lxyzval,))

@nb.extending.lower_getattr(LorentzXYZTType, "mass")
def lower_LorentzXYZT_mass(context, builder, lxyztype, lxyzval):
    return context.compile_internal(builder, lorentz_xyz_mass, nb.float64(lxyztype), (lxyzval,))

And the `__getitem__` access...

In [28]:
@nb.core.typing.templates.infer_global(operator.getitem)
class typer_LorentzXYZT_getitem(nb.core.typing.templates.AbstractTemplate):
    def generic(self, args, kwargs):
        if len(args) == 2 and len(kwargs) == 0 and isinstance(args[0], LorentzXYZTType):
            # Only accept compile-time constants. It's a fair restriction.
            if isinstance(args[1], nb.types.StringLiteral):
                if args[1].literal_value in ("x", "y", "z", "t"):
                    return nb.float64(*args)

In [29]:
@nb.extending.lower_builtin(operator.getitem, LorentzXYZTType, nb.types.StringLiteral)
def lower_getitem_LorentzXYZT(context, builder, sig, args):
    rettype, (lxyztype, wheretype) = sig.return_type, sig.args
    lxyzval, whereval = args

    inproxy = context.make_helper(builder, lxyztype, lxyzval)

    # The value of a StringLiteral is in its compile-time type.
    if wheretype.literal_value == "x":
        return inproxy.x
    elif wheretype.literal_value == "y":
        return inproxy.y
    elif wheretype.literal_value == "z":
        return inproxy.z
    elif wheretype.literal_value == "t":
        return inproxy.t

#### Example

Now we can use all of these. `LorentzXYZTFree` is as fully functional as LorentzXYZT.

In [30]:
@nb.njit
def try_it_out(testit):
    return testit.x, testit["x"], testit.pt, testit.eta, testit.phi, testit.mass

print(try_it_out(testit))

(1.0, 1.0, 2.23606797749979, 1.103586841560145, 1.1071487177940904, 1.4142135623730951)


### Numba: Append to Awkward

Finally, we want to be able to append `LorentzXYZTFree` to a `ArrayBuilder`, as though
it were an attached `LorentzXYZT`. This doesn't need a typer; the types are obvious.

In [31]:
def lower_ArrayBuilder_append_LorentzXYZT(context, builder, sig, args):
    def doit(output, lxyz):
        output.begin_record("LorentzXYZT")
        output.field("x")
        output.real(lxyz.x)
        output.field("y")
        output.real(lxyz.y)
        output.field("z")
        output.real(lxyz.z)
        output.field("t")
        output.real(lxyz.t)
        output.end_record()
    return context.compile_internal(builder, doit, sig, args)

lorentzbehavior["__numba_lower__", ak.ArrayBuilder.append, LorentzXYZTType] = lower_ArrayBuilder_append_LorentzXYZT

#### Example

Attaching free objects to a ArrayBuilder doesn't look any different to the user.

In [32]:
@nb.njit
def fill_it(testit, output):
    output.append(testit)
    output.append(testit)
    
output = ak.ArrayBuilder(behavior=lorentzbehavior)
fill_it(testit, output)

print(output.snapshot())

[Lxyz(1 2 3 4), Lxyz(1 2 3 4)]


### Numba: Addition in Awkward

Now that we have free Lorentz vectors, we can finally define addition.

In [33]:
def typer_lorentz_xyz_add(binop, left, right):
    return LorentzXYZTType()(left, right)

def lower_lorentz_xyz_add(context, builder, sig, args):
    def compute(left, right):
        return LorentzXYZTFree(left.x + right.x, left.y + right.y, left.z + right.z, left.t + right.t)
    return context.compile_internal(builder, compute, sig, args)

In [34]:
lorentzbehavior["__numba_typer__", "LorentzXYZT", operator.add, "LorentzXYZT"] = typer_lorentz_xyz_add
lorentzbehavior["__numba_lower__", "LorentzXYZT", operator.add, "LorentzXYZT"] = lower_lorentz_xyz_add

#### Example

Test it...

In [35]:
@nb.njit
def test_add(input):
    for muons in input:
        for i in range(len(muons)):
            for j in range(i + 1, len(muons)):
                return muons[i] + muons[j]

example5 = ak.Array(example, behavior=lorentzbehavior)

print(test_add(example5))

Lxyz(-15.2 -11 -19.5 94.2)


## Complete example

All together now!

In [36]:
@nb.njit
def do_cool_stuff(input, output):
    for muons in input:
        output.begin_list()

        for i in range(len(muons)):
            output.begin_list()

            for j in range(i + 1, len(muons)):
                zboson = muons[i] + muons[j]

                output.begin_tuple(2)
                output.index(0)
                output.append(zboson)
                output.index(1)
                output.append(zboson.mass)
                output.end_tuple()

            output.end_list()

        output.end_list()

output = ak.ArrayBuilder(behavior=lorentzbehavior)
do_cool_stuff(example5, output)

print(output.snapshot())

[[[(Lxyz(-15.2 -11 -19.5 94.2), 90.2)], []], [[]], [[(, ... [[]], [[]], [[]], [[]]]
